# 图片颜值测试扩展版

范例描述：提交带人脸的图片，根据百度AI的反馈信息，重新生成一张标志出脸部位置并显示各种信息的图片。

本范例的具体介绍请参考百度AI的文档。https://ai.baidu.com/docs#/Face-Python-SDK/81dd3e06

### 第一步、导入百度AI库

In [ ]:
# coding: 'utf-8'
import base64
from aip import AipFace
import math

try:
    from PIL import Image, ImageDraw, ImageFont
except ImportError:
    print("Pillow库没有安装，请在命令提示符中运行:pip install Pillow 完成安装。")
    exit(0)
from matplotlib import pyplot as plt
%matplotlib inline

### 第二步、设置认证信息

注：这里用的是统一的测试账号，有访问次数的限制。请尽量使用自己的账号信息。

In [ ]:
""" 你的 APPID AK SK """
APP_ID = '14563606'
API_KEY = 'ijoqlG1PdSsdxtXc7DNn68jh'
SECRET_KEY = 'Erpr184wiWCG7ZZglFKKG3Zm3up6xUmi '

### 第三步、调用接口提交图片

In [ ]:
client = AipFace(APP_ID, API_KEY, SECRET_KEY)
# 要识别的图片路径
imagefile = 'gyy.jpg'
# image "取决于image_type参数，传入BASE64字符串或URL字符串或FACE_TOKEN字符串"
image = base64.b64encode(open(imagefile, 'rb').read()).decode()
imageType = "BASE64"

""" 如果有可选参数 """
options = {}
options["face_field"] = "age,beauty,expression,gender,glasses"
options["max_face_num"] = 2
options["face_type"] = "LIVE"
options["liveness_control"] = "LOW"

""" 带参数调用人脸检测 """
ret_value = client.detect(image, imageType, options)
print(ret_value)

### 第四步、读取反馈信息

百度AI返回的json信息，Python将解析为字典，用关键字（如result、face_list等）读出需要的信息。

In [ ]:
gender = '女' if ret_value['result']['face_list'][0]['gender']['type'] == 'female' else '男'
gender_probability = ret_value['result']['face_list'][0]['gender']['probability']
age = ret_value['result']['face_list'][0]['age']
beauty = ret_value['result']['face_list'][0]['beauty']
expression = ret_value['result']['face_list'][0]['expression']['type']
dict_expression = {'none': '不笑', 'smile': '微笑', 'laugh': '大笑'}
expression = dict_expression.get(expression, '无')
expression_probability = ret_value['result']['face_list'][0]['expression']['probability']
dict_glasses = {'none': '无眼镜', 'common': '普通眼镜', 'sun': '墨镜'}
glasses = ret_value['result']['face_list'][0]['glasses']['type']
glasses = dict_glasses.get(glasses, '无眼镜')
glasses_probability = ret_value['result']['face_list'][0]['glasses']['probability']

rotate = ret_value['result']['face_list'][0]['location']['rotation']
x1 = ret_value['result']['face_list'][0]['location']['left']
y1 = ret_value['result']['face_list'][0]['location']['top']
width = ret_value['result']['face_list'][0]['location']['width']
height = ret_value['result']['face_list'][0]['location']['height']
theta = rotate * math.pi / 180  # 根据平台返回的角度进行弧度转换
c, s = math.cos(theta), math.sin(theta)  # 利用弧度，进行三角函数处理
im = Image.open(imagefile)
draw = ImageDraw.Draw(im)

### 第五步、标注图片信息

利用三角函数的关系，进行画图。

In [ ]:
firstdot = x1, y1
nextdot = firstdot[0] + width * c, firstdot[1] + width * s
draw.line([firstdot, nextdot], fill=(255, 0, 0, 128), width=3)
firstdot = nextdot
nextdot = firstdot[0] - height * s, firstdot[1] + height * c
draw.line([firstdot, nextdot], fill=(255, 0, 0, 128), width=3)
firstdot = nextdot
nextdot = firstdot[0] - width * c, firstdot[1] - width * s
draw.line([firstdot, nextdot], fill=(255, 0, 0, 128), width=3)
firstdot = nextdot
nextdot = firstdot[0] + height * s, firstdot[1] - height * c
draw.line([firstdot, nextdot], fill=(255, 0, 0, 128), width=3)
# 产生新的图片
pic_width, pic_height = im.size
img2 = Image.new('RGB', (pic_width + 400, pic_height), (255, 255, 255))
img2.paste(im.crop(),(0, 0, pic_width, pic_height))

In [ ]:
## 在图片上写入信息
format_str = '性别：{0}(可靠度：{1})\n' \
             '年龄：{2}\n' \
             '颜值：{3}\n' \
             '表情：{4}(可靠度：{5})\n' \
             '佩戴：{6}(可靠度：{7})'
print_str = format_str.format(gender, gender_probability, age,
                              beauty, expression,
                              expression_probability, glasses,
                              glasses_probability)
font = ImageFont.truetype("simhei.ttf", 21, index=0)
d = ImageDraw.Draw(img2)
d.text([pic_width + 5, 60], print_str, font=font, fill=(255, 0, 0))

# 保存图片
newimagefile='{}.png'.format(imagefile+'识别结果')
img2.save(newimagefile)
print(print_str)

### 第六步、显示图片

利用matplotlib在网页上显示图片。查看这个文件夹，看看是不是多了一个名为“gyy.jpg识别结果.png”的图片？

In [ ]:
plt.imshow(plt.imread(imagefile))
plt.axis('off') #不显示坐标
plt.show()

plt.imshow(plt.imread(newimagefile))
plt.axis('off') #不显示坐标
plt.show()